# Prophet e Random Forest

In [69]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from prophet import Prophet

/home/jose/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [51]:
# Carregar os datasets
data_base_tratada = pd.read_excel('../baseDeDados/dataBaseTratada.xlsx')
data_base_teste = pd.read_excel('../baseDeDados/dataBaseTeste.xlsx')

In [52]:
# Converter a coluna para o formato datetime
data_base_tratada['order_approved_at'] = pd.to_datetime(data_base_tratada['order_approved_at'])

# Extrair o dia da semana
data_base_tratada['dia_da_semana'] = data_base_tratada['order_approved_at'].dt.day_name()

# Criar colunas binárias para cada dia da semana
data_base_tratada = pd.get_dummies(data_base_tratada, columns=['dia_da_semana'])

# Converter colunas binárias para 1 e 0
for col in data_base_tratada.columns:
    if 'dia_da_semana_' in col:
        data_base_tratada[col] = data_base_tratada[col].astype(int)

In [53]:
# Converter a coluna para o formato category
data_base_tratada['payment_type'] = data_base_tratada['payment_type'].astype('category')

# Criar colunas binárias para cada tipo de pagamento
data_base_tratada = pd.get_dummies(data_base_tratada, columns=['payment_type'])

# Converter colunas binárias para 1 e 0
for col in data_base_tratada.columns:
    if 'payment_type' in col:
        data_base_tratada[col] = data_base_tratada[col].astype(int)

In [54]:
# Converter a coluna para o formato datetime
data_base_teste['order_approved_at'] = pd.to_datetime(data_base_teste['order_approved_at'])

# Extrair o dia da semana
data_base_teste['dia_da_semana'] = data_base_teste['order_approved_at'].dt.day_name()

# Criar colunas binárias para cada dia da semana
data_base_teste = pd.get_dummies(data_base_teste, columns=['dia_da_semana'])

# Converter colunas binárias para 1 e 0
for col in data_base_teste.columns:
    if 'dia_da_semana_' in col:
        data_base_teste[col] = data_base_teste[col].astype(int)

In [55]:
# Converter a coluna para o formato category
data_base_teste['payment_type'] = data_base_teste['payment_type'].astype('category')

# Criar colunas binárias para cada tipo de pagamento
data_base_teste = pd.get_dummies(data_base_teste, columns=['payment_type'])

# Converter colunas binárias para 1 e 0
for col in data_base_teste.columns:
    if 'payment_type' in col:
        data_base_teste[col] = data_base_teste[col].astype(int)

In [65]:
# (Assumindo que o DELIVERY_TIME é a última coluna do dataBaseTratada)
X = data_base_tratada[['customer_zip_code_prefix', 'price', 'freight_value', 'payment_value', 'payment_installments', 'payment_type_boleto', 'payment_type_credit_card', 'payment_type_debit_card', 'payment_type_voucher','product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'seller_zip_code_prefix', 'Distancia', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'dia_da_semana_Friday', 'dia_da_semana_Monday', 'dia_da_semana_Saturday', 'dia_da_semana_Sunday', 'dia_da_semana_Thursday', 'dia_da_semana_Tuesday', 'dia_da_semana_Wednesday']]
y = data_base_tratada[['delivery_time']]

# Dividir o dataset tratado em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo Random Forest
model = RandomForestRegressor(n_estimators=40, random_state=42)
model.fit(X_train, y_train)

# Avaliar o modelo no conjunto de validação
y_pred_val = model.predict(X_val)
mae_val = mean_absolute_error(y_val, y_pred_val)
print(f'MAE no conjunto de validação: {mae_val}')

/home/jose/.local/lib/python3.10/site-packages/sklearn/base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


MAE no conjunto de validação: 4.394203294591276


In [66]:
# Aplicar o modelo ao conjunto de teste
X_test = data_base_teste[['customer_zip_code_prefix', 'price', 'freight_value', 'payment_value', 'payment_installments', 'payment_type_boleto', 'payment_type_credit_card', 'payment_type_debit_card', 'payment_type_voucher','product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'seller_zip_code_prefix', 'Distancia', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'dia_da_semana_Friday', 'dia_da_semana_Monday', 'dia_da_semana_Saturday', 'dia_da_semana_Sunday', 'dia_da_semana_Thursday', 'dia_da_semana_Tuesday', 'dia_da_semana_Wednesday']]
y_pred_test = model.predict(X_test)

# Avaliar o modelo no conjunto de validação
y_pred_val = model.predict(X_val)
mae_val = mean_absolute_error(y_val, y_pred_val)
print(f'MAE no conjunto de validação: {mae_val}')

MAE no conjunto de validação: 4.394203294591276


In [71]:
# Preparar os dados para o Prophet
prophet_data = data_base_tratada[['order_purchase_timestamp', 'delivery_time']].rename(columns={'order_purchase_timestamp': 'ds', 'delivery_time': 'y'})

# Instanciar e treinar o modelo Prophet
prophet_model = Prophet()
prophet_model.fit(prophet_data)

# Fazer previsões com o modelo Prophet
future = prophet_model.make_future_dataframe(periods=len(data_base_teste))
forecast = prophet_model.predict(future)

# Avaliar a performance do Prophet
y_test_prophet = data_base_teste['delivery_time'].values
y_pred_prophet = forecast['yhat'][-len(data_base_teste):].values
mae_test_prophet = mean_absolute_error(y_test_prophet, y_pred_prophet)
print(f'MAE no conjunto de teste com Prophet: {mae_test_prophet}')

# Salvar as previsões do Prophet no dataset de teste
data_base_teste['previsaoFinal'] = y_pred_prophet

21:40:17 - cmdstanpy - INFO - Chain [1] start processing
21:40:40 - cmdstanpy - INFO - Chain [1] done processing


In [67]:
# Salvar as previsões no dataset de teste
data_base_teste['previsaoFinal'] = y_pred_test

# Enviando para um dataFrame do Kaggle
dfKaggle = data_base_teste[['order_id', 'previsaoFinal']]

# Salvar os resultados
dfKaggle.to_csv('../resultadoPrevisoes/previsaoProphetRandomForest.csv', index=False)

In [68]:
dfKaggle

,order_id,previsoes
0,cee3292f46ede6ea1dfabfcb200fcf47,17.863553
1,50dca53ca33b739bef09e7933e8b380e,11.700286
2,8087ec71e393d4dc6fc48041fe63cd51,6.984536
3,e6b6557ce111de79b31cc857f20ba212,12.402936
4,0b09c5e4c2512f627190ac55a78c35a3,15.386702
...,...,...
28147,81cedc4d2132eb6f512354166e51b187,13.801536
28148,95811675a89b66659996da5285a021db,12.971069
28149,25bd2ad9d282f59e4e6a048ef9bf214f,22.044170
28150,9f9ac312604229686c04260955495c70,8.260170
